# Proyecto Final: Integración de 3 o más cuerpos

- Carlos Daniel López Mimenza (**@SirSilverCharles**)
- Daniel Martínez Urrieta (**@danmarurr**)
- María Fernanda Pérez Ramírez (**@FernandaPerez**)

Para el problema de $N$ cuerpos, consideremos las ecuaciones de movimiento para cada una ellas (con la posición $\vec{q}_i$, masa $m_i$), suponiendo que las partículas se encuentran en el mismo plano (2D):

\begin{align}
    m_i \frac{{\rm d}^2\vec{q}_i}{{\rm d}t^2} = \sum_{i \neq j} \frac{Gm_im_j}{||\vec{q}_i - \vec{q}_j||^3}(\vec{q}_i - \vec{q}_j)
\end{align}

## Caso más simple: Tres cuerpos.

Para éste caso, tendremos que las ecuaciones que describen las trayectorias de las partículas son:

\begin{align}
    \ddot{\mathbf{x}}_1 &= - \frac{Gm_2}{(x_1 - x_2)^3}(\mathbf{x}_1 - \mathbf{x}_2) - \frac{Gm_3}{(x_1 - x_3)^3}(\mathbf{x}_1 - \mathbf{x}_3)\\
    \ddot{\mathbf{x}}_2 &= - \frac{Gm_3}{(x_2 - x_3)^3}(\mathbf{x}_2 - \mathbf{x}_3) - \frac{Gm_1}{(x_2 - x_1)^3}(\mathbf{x}_2 - \mathbf{x}_1)\\
    \ddot{\mathbf{x}}_3 &= - \frac{Gm_1}{(x_3 - x_1)^3}(\mathbf{x}_3 - \mathbf{x}_1) - \frac{Gm_2}{(x_3 - x_2)^3}(\mathbf{x}_3 - \mathbf{x}_2)
\end{align}

Para tomar sólo ecuaciones de primer orden (y así usar Taylor) implementamos además que:
\begin{align}
    \dot{\mathbf{v}}_j = \ddot{\mathbf{x}}_j \ \ \Rightarrow \dot{\mathbf{x}}_j = \mathbf{v}_j
\end{align}

In [1]:
#Agrego Línea para instalar modulo de Taylor (cortesia de Luis)
Pkg.add("TaylorSeries")

INFO: Cloning cache of Docile from git://github.com/MichaelHatherly/Docile.jl.git
INFO: Cloning cache of FactCheck from git://github.com/JuliaLang/FactCheck.jl.git
INFO: Cloning cache of TaylorSeries from git://github.com/lbenet/TaylorSeries.jl.git
INFO: Installing Docile v0.5.23
INFO: Installing FactCheck v0.4.2
INFO: Installing TaylorSeries v0.1.1
INFO: Package database updated
INFO: METADATA is out-of-date — you may not have the latest version of TaylorSeries
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [34]:
typeof(x)

TaylorSeries.Taylor1{Int64}

In [3]:
using TaylorSeries

In [13]:
function paso_int{T<:Real}(x_0::Taylor1{T}, p::Int)
    h1 = 0.5*(eps(1.0)/abs(x_0.coef[p + 1]))^(1/p) 
    h2 = 0.5*(eps(1.0)/abs(x_0.coef[p]))^(1/(p-1))
    minimum([h1, h2])
end

function Horner{T<:Real, S<:Real}(x_0::Taylor1{S}, h::T)
    n = x_0.order
    suma = zeros(n)
    suma[1] = x_0.coeffs[n]
    for j in 2:n
        suma[j] = x_0.coeffs[n + 1 - j] + h*suma[j - 1]
    end
    suma[n]
end

function Integrador_1erIntento(pos_iniciales::Array{Array{Float64,1},1}, vel_ini::Array{Array{Float64,1},1},
    masas::Array{Float64,1}, t0::Float64, tf:Float64, p::Int)
    #Para ahorrar al momento de escribir los nombres se indicarán así: pi donde i es el número de cuerpo,
    # y p inidica si es posición o velocidad
    #Creamos los arreglos con los tiempos y las soluciones
    v1 = typeof(vel_ini[1])[vel_ini[1]]
    v2 = typeof(vel_ini[2])[vel_ini[2]]
    v3 = typeof(vel_ini[3])[vel_ini[3]]
    p1 = typeof(pos_iniciales[1])[pos_iniciales[1]]
    p2 = typeof(pos_iniciales[2])[pos_iniciales[2]]
    p3 = typeof(pos_iniciales[3])[pos_iniciales[3]]
    ts = Float64[t0] #arreglo inicial para los tiempos
    
    m1 = masas[1]
    m2 = masas[2]
    m3 = masas[3]
    
    t = t0           
    x1, y1, z1 = p1[1]
    x2, y2, z2 = p2[1]
    x3, y3, z3 = p3[1]
    vx1, vy1, vz1 = v1[1]
    vx2, vy2, vz2 = v2[1]
    vx3, vy3, vz3 = v3[1]       
    
    while t <= tf
        #Creo arreglos de cada parámetro
        arr_x1 = Float64[x1]
        arr_x2 = Float64[x2]
        arr_x3 = Float64[x3]
        arr_y1 = Float64[y1]
        arr_y2 = Float64[y2]
        arr_y3 = Float64[y3]
        arr_z1 = Float64[z1]
        arr_z2 = Float64[z2]
        arr_z3 = Float64[z3]
        
        arr_vx1 = Float64[vx1]
        arr_vx2 = Float64[vx2]
        arr_vx3 = Float64[vx3]
        arr_vy1 = Float64[vy1]
        arr_vy2 = Float64[vy2]
        arr_vy3 = Float64[vy3]
        arr_vz1 = Float64[vz1]
        arr_vz2 = Float64[vz2]
        arr_vz3 = Float64[vz3]
        
        #Creo Taylor's para cada parámetro
        for j in 1:p
            Taylor_arr_x1 = Taylor1(arr_x1)
            Taylor_arr_x2 = Taylor1(arr_x2)
            Taylor_arr_x3 = Taylor1(arr_x3)
            Taylor_arr_y1 = Taylor1(arr_y1)
            Taylor_arr_y2 = Taylor1(arr_y2)
            Taylor_arr_y3 = Taylor1(arr_y3)
            Taylor_arr_z1 = Taylor1(arr_z1)
            Taylor_arr_z2 = Taylor1(arr_z2)
            Taylor_arr_z3 = Taylor1(arr_z3)
            Taylor_arr_vx1 = Taylor1(arr_vx1)
            Taylor_arr_vx2 = Taylor1(arr_vx2)
            Taylor_arr_vx3 = Taylor1(arr_vx3)
            Taylor_arr_vy1 = Taylor1(arr_vy1)
            Taylor_arr_vy2 = Taylor1(arr_vy2)
            Taylor_arr_vy3 = Taylor1(arr_vy3)
            Taylor_arr_vz1 = Taylor1(arr_vz1)
            Taylor_arr_vz2 = Taylor1(arr_vz2)
            Taylor_arr_vz3 = Taylor1(arr_vz3)
            #Calculamos las normas de los vectores
            r1 = (Taylor_arr_x1^2 + Taylor_arr_y1^2 + Taylor_arr_z1^2)^(1/2)
            r2 = (Taylor_arr_x2^2 + Taylor_arr_y2^2 + Taylor_arr_z2^2)^(1/2)
            r3 = (Taylor_arr_x3^2 + Taylor_arr_y3^2 + Taylor_arr_z3^2)^(1/2)
            
            ##Definimos la operación de las 9 ecs de movimiento.
            #Para el cuerpo 1
            fx1 = -m2(Taylor_arr_x1 - Taylor_arr_x2)/(r1 - r2)^3 - m3(Taylor_arr_x1 - Taylor_arr_x3)/(r1 - r3)^3
            fy1 = -m2(Taylor_arr_y1 - Taylor_arr_y2)/(r1 - r2)^3 - m3(Taylor_arr_y1 - Taylor_arr_y3)/(r1 - r3)^3
            fz1 = -m2(Taylor_arr_z1 - Taylor_arr_z2)/(r1 - r2)^3 - m3(Taylor_arr_z1 - Taylor_arr_z3)/(r1 - r3)^3
            #Para el cuerpo 2
            fx2 = -m3(Taylor_arr_x2 - Taylor_arr_x3)/(r2 - r3)^3 - m1(Taylor_arr_x2 - Taylor_arr_x1)/(r2 - r1)^3
            fy2 = -m3(Taylor_arr_y2 - Taylor_arr_y3)/(r2 - r3)^3 - m1(Taylor_arr_y2 - Taylor_arr_y1)/(r2 - r1)^3
            fz2 = -m3(Taylor_arr_z2 - Taylor_arr_z3)/(r2 - r3)^3 - m1(Taylor_arr_z2 - Taylor_arr_z1)/(r2 - r1)^3
            #Para el cuerpo 3
            fx3 = -m1(Taylor_arr_x3 - Taylor_arr_x1)/(r3 - r1)^3 - m3(Taylor_arr_x3 - Taylor_arr_x1)/(r3 - r1)^3
            fy3 = -m1(Taylor_arr_y3 - Taylor_arr_y1)/(r3 - r1)^3 - m3(Taylor_arr_y3 - Taylor_arr_y1)/(r3 - r1)^3
            fz3 = -m1(Taylor_arr_z3 - Taylor_arr_z1)/(r3 - r1)^3 - m3(Taylor_arr_z3 - Taylor_arr_z1)/(r3 - r1)^3
            
            #Incluimos los nuevos coeficientes
            arr_x1 = push!(arr_x1, Taylor_arr_vx1.coeffs[j]/j)
            arr_x2 = push!(arr_x2, Taylor_arr_vx2.coeffs[j]/j)
            arr_x3 = push!(arr_x3, Taylor_arr_vx3.coeffs[j]/j)
            arr_y1 = push!(arr_y1, Taylor_arr_vy1.coeffs[j]/j)
            arr_y2 = push!(arr_y2, Taylor_arr_vy2.coeffs[j]/j)
            arr_y3 = push!(arr_y3, Taylor_arr_vy3.coeffs[j]/j)
            arr_z1 = push!(arr_z1, Taylor_arr_vz1.coeffs[j]/j)
            arr_z2 = push!(arr_z2, Taylor_arr_vz2.coeffs[j]/j)
            arr_z3 = push!(arr_z3, Taylor_arr_vz3.coeffs[j]/j)
            arr_vx1 =push!(arr_vx1, fx1.coeffs[j]/j)
            arr_vx2 =push!(arr_vx2, fx2.coeffs[j]/j)
            arr_vx3 =push!(arr_vx3, fx3.coeffs[j]/j)
            arr_vy1 =push!(arr_vy1, fy1.coeffs[j]/j)
            arr_vy2 =push!(arr_vy2, fy2.coeffs[j]/j)
            arr_vy3 =push!(arr_vy3, fy3.coeffs[j]/j)
            arr_vz1 =push!(arr_vz1, fz1.coeffs[j]/j)
            arr_vz2 =push!(arr_vz2, fz2.coeffs[j]/j)
            arr_vz3 =push!(arr_vz3, fz3.coeffs[j]/j)
        end
        Taylor_arr_x1 = Taylor1(arr_x1)
        Taylor_arr_x2 = Taylor1(arr_x2)
        Taylor_arr_x3 = Taylor1(arr_x3)
        Taylor_arr_y1 = Taylor1(arr_y1)
        Taylor_arr_y2 = Taylor1(arr_y2)
        Taylor_arr_y3 = Taylor1(arr_y3)
        Taylor_arr_z1 = Taylor1(arr_z1)
        Taylor_arr_z2 = Taylor1(arr_z2)
        Taylor_arr_z3 = Taylor1(arr_z3)
        Taylor_arr_vx1 = Taylor1(arr_vx1)
        Taylor_arr_vx2 = Taylor1(arr_vx2)
        Taylor_arr_vx3 = Taylor1(arr_vx3)
        Taylor_arr_vy1 = Taylor1(arr_vy1)
        Taylor_arr_vy2 = Taylor1(arr_vy2)
        Taylor_arr_vy3 = Taylor1(arr_vy3)
        Taylor_arr_vz1 = Taylor1(arr_vz1)
        Taylor_arr_vz2 = Taylor1(arr_vz2)
        Taylor_arr_vz3 = Taylor1(arr_vz3)
        
        #Calculamos todas las h's posibles        
        h1 = paso_int(Taylor_arr_x1,p)
        h2 = paso_int(Taylor_arr_x2,p)
        h3 = paso_int(Taylor_arr_x3,p)
        h4 = paso_int(Taylor_arr_y1,p)
        h5 = paso_int(Taylor_arr_y2,p)
        h6 = paso_int(Taylor_arr_y3,p)
        h7 = paso_int(Taylor_arr_z1,p)
        h8 = paso_int(Taylor_arr_z2,p)
        h9 = paso_int(Taylor_arr_z3,p)
        h10 = paso_int(Taylor_arr_vx1,p)
        h11 = paso_int(Taylor_arr_vx2,p)
        h12 = paso_int(Taylor_arr_vx3,p)
        h13 = paso_int(Taylor_arr_vy1,p)
        h14 = paso_int(Taylor_arr_vy2,p)
        h15 = paso_int(Taylor_arr_vy3,p)
        h16 = paso_int(Taylor_arr_vz1,p)
        h17 = paso_int(Taylor_arr_vz2,p)
        h18 = paso_int(Taylor_arr_vz3,p)
        
        #Elegimos el h más pequeño
        h = minimum([h1, h2, h3, h4, h5, h6, h7, h8, h9, h10, h11, h12, h13, h14, h15, h16, h17, h18])
        t += h
        
        #Calculamos el siguiente paso usando Horner
        x1
        x2
        x3
        y1
        y2
        y3
        z1
        z2
        z3
        vx1
        vx2
        vx3
        vy1
        vy2
        vy3
        vz1
        vz2
        vz3
        
        
        x = Horner(xt, h)
        v = Horner(vt, h)
        #@show v, x
        #Lo siguiente lo hacemos para evitar que la solución tenga valores que no queremos,
        #pues si éstos aparecen, obliga a detener el proceso
        #if isnan(x) | (x == Inf) | (x == -Inf)
        #    return ts, xs
        #    break
        #end
        vs = push!(vs, v)
        xs = push!(xs, x)
        ts = push!(ts, t)
    end
    ts, xs, vs #devuelve tiempo, posición y velocidad.
end

Integrador_1erIntento (generic function with 1 method)

In [24]:
TaylorN(b)

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{TaylorSeries.TaylorN{T<:Number}}, ::Array{Array{Int64,1},1})
This may have arisen from a call to the constructor TaylorSeries.TaylorN{T<:Number}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T<:Number}(::Type{T<:Number}, !Matched::Char)
  convert{T<:Number}(::Type{T<:Number}, !Matched::Base.Dates.Period)
  ...
while loading In[24], in expression starting on line 1